<a href="https://colab.research.google.com/github/lucifernandes/acidentesPRF/blob/master/scrub_explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados abertos sobre Acidentes da Polícia Rodoviária Federal

##Etapa Limpeza

##Etapa Exploração